In [1]:
import requests
import pandas as pd
import time
import networkx as nx

# Get ORF interaction information from Saccharomyces Genome Database 

In [2]:
#list of ORFs obtained from SGD, read in as dataframe

orfdf = pd.read_csv(open("SGD_ORF.txt", 'rb'), sep='\t')

#orf names placed in list for ease
orfs = list(orfdf["Feature Name"])

#for each ORF, regulators are obtained from server and placed into dataframe

SGD_BASE_URL = 'https://www.yeastgenome.org/webservice'


In [ ]:
#for each ORF, obtain interactors from server and place into dataframe. Label each interactor by their interaction type

time0 = time.time()
interactors1 = []
interactors2 = []
interactortype = []
i=0
for orf in orfs:
    
    url = 'https://www.yeastgenome.org/backend/locus/' + orf + '/interaction_details'
    go = True
    while go == True:
        try:
            response = requests.get(url=url).json()
            go = False
        except ValueError:
            print("decode error")
            time0 = time.time()
            while time.time()-time0<1:
                a=1
            pass
    orfinteractors1 = []
    orfinteractors2 = []
    orfinteractortype = []
    for value in response:
        orfinteractors1.append(value['locus1']['format_name'])
        orfinteractors2.append(value['locus2']['format_name'])
        if "Genetic" in value['experiment']['display_name']:
            orfinteractortype.append(value['experiment']['display_name'])
        else:
            orfinteractortype.append(value['interaction_type'])
    interactors1.append(orfinteractors1)
    interactors2.append(orfinteractors2)
    interactortype.append(orfinteractortype)
    time0 = time.time()
    i+=1
    while time.time()-time0<1:
        a=1
    
    
orfdf["interactors_type"] = interactortype
orfdf['interactors1'] = interactors1
orfdf['interactors2'] = interactors2

featurename = list(orfdf['Feature Name'])
allinteractors = []
for i in range(0,len(interactors1)):
    orfinteractors = []
    for j in range(0,len(interactors1[i])):
        test = 0
        if interactors1[i][j] != featurename[i]:
            orfinteractors.append(interactors1[i][j])
            test+=1
        if interactors2[i][j] != featurename[i]:
            orfinteractors.append(interactors2[i][j])
            test+=1
        if test > 1:
            print("Error at i:" + str(i)+ " j:" + str(j))
    allinteractors.append(orfinteractors)
orfdf['interactors'] = allinteractors
orfdf.to_pickle("orfdf_both_interactions.pickle")

In [3]:
orfdf = pd.read_pickle("orfdf_both_interactions.pickle")

# Combine data obtained from reactome into singe dataframe

In [4]:
#place enzyme reactions into dataframe

categories = []
with open('enzrxns.dat') as file:
    for line in file:
        if line.strip()[0] != '#' and line.strip()[0] != r'/' and line.strip()[0] != '^':
            categories.append(line.strip().split(" - ")[0])

categories = list(set(categories))

entry = -1
table = []
for value in categories:
    table.append([])
with open('enzrxns.dat') as file:
    for line in file:
        #print(line)
        if line.strip()[0] == r'/'or entry == -1:
            if len(line.strip())>1:
                if line.strip()[1] == r'/':
                    entry += 1
                    for value in table:
                        value.append([])
        elif line.strip()[0] != '#' and line.strip()[0] != r'/' and line.strip()[0] != '^':
            table[categories.index(line.strip().split(" - ")[0])][entry].append(line.strip().split(" - ")[1])
            
enzrxnsdf = pd.DataFrame(table)
enzrxnsdf = enzrxnsdf.transpose()

enzrxnsdf.columns = categories

enzrxnsdf.to_csv("enzreactions.csv")
enzrxnsdf.to_pickle("enzreactions.pickle")

In [5]:
#place chemical reactions into dataframe

categories = []
with open('reactions.dat') as file:
    for line in file:
        if line.strip()[0] != '#' and line.strip()[0] != r'/' and line.strip()[0] != '^':
            categories.append(line.strip().split(" - ")[0])
categories = list(set(categories))
entry = -1
table = []
for value in categories:
    table.append([])
with open('reactions.dat') as file:
    for line in file:
        #print(line)
        if line.strip()[0] == r'/'or entry == -1:
            if len(line.strip())>1:
                if line.strip()[1] == r'/':
                    entry += 1
                    for value in table:
                        value.append([])
        elif line.strip()[0] != '#' and line.strip()[0] != r'/' and line.strip()[0] != '^':
            table[categories.index(line.strip().split(" - ")[0])][entry].append(line.strip().split(" - ")[1])
            
reactionsdf = pd.DataFrame(table)
reactionsdf = reactionsdf.transpose()

reactionsdf.columns = categories

reactionsdf.to_csv("reactions.csv")
reactionsdf.to_pickle("reactions.pickle")

In [6]:
#get compounds

categories = []
with open('compounds.dat') as file:
    for line in file:
        if line.strip()[0] != '#' and line.strip()[0] != r'/' and line.strip()[0] != '^':
            categories.append(line.strip().split(" - ")[0])
categories = list(set(categories))
entry = -1
table = []
for value in categories:
    table.append([])
with open('compounds.dat') as file:
    for line in file:
        #print(line)
        if line.strip()[0] == r'/'or entry == -1:
            if len(line.strip())>1:
                if line.strip()[1] == r'/':
                    entry += 1
                    for value in table:
                        value.append([])
        elif line.strip()[0] != '#' and line.strip()[0] != r'/' and line.strip()[0] != '^':
            if len(line.strip().split(" - ")) > 1:
                table[categories.index(line.strip().split(" - ")[0])][entry].append(line.strip().split(" - ")[1])
            
compoundsdf = pd.DataFrame(table)
compoundsdf = compoundsdf.transpose()

compoundsdf.columns = categories

compoundsdf.to_csv("compounds.csv")
compoundsdf.to_pickle("compounds.pickle")

In [7]:
#put enzymes into df, split multimeric enyzmes into monomers to identify each protein for each reaction

enzymesdf = pd.read_csv(open("enzymes.col.txt", 'rb'), sep='\t')

subunits = list(enzymesdf['SUBUNIT-COMPOSITION'])

i = 0
while i < len(subunits):
    if (type(subunits[i]) == type("")):
        subunitslist = subunits[i].split(",")
        j = 0
        while j < len(subunitslist):
            if "-MONOMER" in subunitslist[j]: 
                subunitslist[j]=subunitslist[j].split("-MONOMER")[0]
            subunitslist[j]=subunitslist[j].split('*')[1]
            j+=1
        subunits[i] = subunitslist
    else:
        subunits[i] = []
    i+=1
    
    
enzymesdf['subunits'] = subunits
enzymerxnid = enzymesdf["UNIQUE-ID"]

genesdf = pd.read_csv(open("genes.col.txt", 'rb'), sep='\t')
geneid = list(genesdf["UNIQUE-ID"])

#combine chemical reactions with enzymes

allgene_enzymerxns = []
i = 0
while i < len(geneid):
    j = 0
    gene_enzymerxns = []
    while j < len(subunits):
        k=0
        while k < len(subunits[j]):
            if geneid[i] == subunits[j][k]:
                gene_enzymerxns.append(enzymerxnid[j])
            k+=1
        j+=1
    allgene_enzymerxns.append(gene_enzymerxns)
    i+=1
    
genesdf["Enzyme Reactions"] = allgene_enzymerxns


right = list(reactionsdf["RIGHT"])
left = list(reactionsdf["LEFT"])
reactionsID = list(reactionsdf["UNIQUE-ID"])
enzymereactions = list(reactionsdf["ENZYMATIC-REACTION"])

i = 0
finalright = []
finalleft = []
finalreactionsID = []
while i < len(allgene_enzymerxns):
    j = 0
    tempright = []
    templeft = []
    tempreact = []
    while j < len(allgene_enzymerxns[i]):
        k = 0
        while k < len(enzymereactions):
            l = 0
            while l < len(enzymereactions[k]):
                if allgene_enzymerxns[i][j] == enzymereactions[k][l]:
                    tempright.append(right[k])
                    templeft.append(left[k])
                    tempreact.append(reactionsID[k])
                    
                l+=1
            k+=1
        j+=1
    finalright.append(tempright)
    finalleft.append(templeft)
    finalreactionsID.append(tempreact)
    i+=1
    
genesdf["ReactionID"] = finalreactionsID
genesdf["Left"] = finalleft
genesdf["Right"] = finalright

genesdf.to_csv("genesdf.csv")
genesdf.to_pickle("genesdf.pickle")

# Combine SGD dataframe and Reactome dataframe

In [8]:
#combine dataframes

orfs = list(orfdf["Feature Name"])
names = list(orfdf["Standard Gene Name"])
alias = list(orfdf["Alias"])
#regulators = list(orfdf["Regulators"])
#yeastract = list(orfdf["yeastract"])
interactors = list(orfdf["interactors"])
interactors_type = list(orfdf["interactors_type"])
#interactome = list(orfdf["interactome"])

names[609]="OCT1"

genesdfname = genesdf['NAME']
left = genesdf['Left']
right = genesdf['Right']

newleft = []
newright = []

for value in orfs:
    newleft.append(0)
    newright.append(0)
    
i = 0
while i < len(names):
    if type(names[i]) != type(""):
        names[i] = orfs[i]
        if (type(names[i]) != type(1.0)):
            i=i
            #print(names[i])
    i+=1

i = 0
while i < len(alias):
    if type(alias[i]) != type(""):
        alias[i] = ""
    i+=1
    
i = 0
while i < len(genesdfname):
    found = False
    j = 0
    while j < len(orfs):
        if genesdfname[i] == orfs[j] or genesdfname[i] == names[j] or (genesdfname[i] in alias[j].split("|")):
            newleft[j] = left[i]
            newright[j] = right[i]
            found = True
            j=len(orfs)
        j+=1
    if found == False:
        orfs.append(genesdfname[i])
        names.append(genesdfname[i])
        alias.append(genesdfname[i])
        #regulators.append([])
        #yeastract.append([])
        interactors.append([])
        interactors_type.append([])
        #interactome.append([])
        newleft.append(left[i])
        newright.append(right[i])
        #print("cant find" + str(genesdfname[i]))
    i+=1

In [9]:
connectionsdf = pd.DataFrame(list(zip(names, orfs, alias, interactors, interactors_type, newleft, newright)), 
                              columns =["Name", "ORF", "Alias","SGDInteractors", "InteractorsType", "ReactionLeft", 'ReactionRight'])

In [ ]:
connectionsdf.to_pickle("connectionsdf.pickle")
connectionsdf.to_csv("connectionsdf.csv")


In [ ]:
connectionsdf = pd.read_pickle("connectionsdf.pickle")

In [10]:
orfs = list(connectionsdf["ORF"])
names = list(connectionsdf["Name"])
alias = list(connectionsdf["Alias"])
#regulators = list(connectionsdf["SGDRegulators"])
#yeastract = list(connectionsdf["YeastractRegulators"])
interactors = list(connectionsdf["SGDInteractors"])
interactors_type = list(connectionsdf["InteractorsType"])
#interactome = list(connectionsdf["InteractomeInteractors"])
left = list(connectionsdf["ReactionLeft"])
right = list(connectionsdf["ReactionRight"])

i = 0
while i < len(left):
    if left[i] == 0:
        left[i] = []
    if right[i] == 0:
        right[i] = []
        
    i+=1

# Use dataframe to construct graph linking orfs by metabolic and genetic interactions

In [12]:
G = nx.Graph()
G1 = nx.Graph()
i = 0
G.add_node('CIT', form = 'compound', distance = int(0))
while i < len(orfs):
    try:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = int(10*round(nx.dijkstra_path_length(G1, orfs[i], 'CIT'), 1)))
    except:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = -1)
        
    k = 0
    while k < len(left[i]):
        l = 0
        while l < len(left[i][k]):
            try:
                G.add_node(left[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, left[i][k][l], 'CIT'),1)))
            except:
                G.add_node(left[i][k][l], form = 'compound', distance = -1)
                
            G.add_edge(orfs[i], left[i][k][l], label = "backward_rxn", weight = int(10*0.3))
            #print("add edge")
            l+=1
        k+=1
    
          
    k = 0
    while k < len(right[i]):
        l = 0
        while l < len(right[i][k]):
            try:
                G.add_node(right[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, right[i][k][l], 'CIT'),1)))
            except:
                G.add_node(right[i][k][l], form = 'compound', distance = -1)
            G.add_edge(orfs[i], right[i][k][l], label = "forward_rxn", weight = int(10*0.3))
            #print("ADD EDGE")
            l+=1
        k+=1
    
    j=0
    #print(i/len(orfs))
    #nx.write_graphml(G,"metabolism_test.xml")
    #print (str(i /len(orfs)) + " " + str(i) + " " + str(len(orfs)), end='\r')
    i+=1
G.remove_node("PROTON")
G.remove_node("WATER")
G.remove_node("ATP")
G.remove_node("ADP")
G.remove_node("PPI")
G.remove_node("Pi")
G.remove_node("Protein-L-serine-or-L-threonine")
G.remove_node("Protein-Ser-or-Thr-phosphate")
G.remove_node("AMP")
G.remove_node("NAD")
G.remove_node("NADH")
G.remove_node("CO-A")
G.remove_node("NADP")
G.remove_node("NADPH")
G.remove_node("CARBON-DIOXIDE")
G.remove_node("GLT")
G.remove_node("S-ADENOSYLMETHIONINE")
G.remove_node("OXYGEN-MOLECULE")
G.remove_node("ACETYL-COA")
G.remove_node("AMMONIUM")
G.remove_node("ADENOSYL-HOMO-CYS")
G.remove_node("Nucleoside-Triphosphates")
G.remove_node("Peptides-holder")
G.remove_node("RNA-Holder")
G.remove_node("Cytochromes-C-Oxidized")
G.remove_node("Cytochromes-C-Reduced")
G.remove_node("GDP")
G.remove_node("Ubiquitin-C-Terminal-Glycine")
G.remove_node("General-Protein-Substrates")

i = 0

while i < len(orfs):
    k = 0
    while k < len(interactors[i]):
        if interactors_type[i][k] == "Positive Genetic":
            G.add_edge(names[i], interactors[i][k], label = interactors_type[i][k], weight = int(10))
 
        k+=1

    i+=1
    #print (str(i /len(orfs)) + " " + str(i) + " " + str(len(orfs)), end='\r')

import copy
G1 = copy.deepcopy(G)


G = nx.Graph()
i = 0
G.add_node('CIT', form = 'compound', distance = int(0))
while i < len(orfs):
    try:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = int(10*round(nx.dijkstra_path_length(G1, orfs[i], 'CIT'), 1)))
    except:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = -1)
    k = 0
    while k < len(left[i]):
        l = 0
        while l < len(left[i][k]):
            try:
                G.add_node(left[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, left[i][k][l], 'CIT'),1)))
            except:
                G.add_node(left[i][k][l], form = 'compound', distance = -1)
            G.add_edge(orfs[i], left[i][k][l], label = "backward_rxn", weight = 10*0.3)
            #print("add edge")
            l+=1
        k+=1
    
          
    k = 0
    while k < len(right[i]):
        l = 0
        while l < len(right[i][k]):
            try:
                G.add_node(right[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, right[i][k][l], 'CIT'),1)))
            except:
                G.add_node(right[i][k][l], form = 'compound', distance = -1)
            G.add_edge(orfs[i], right[i][k][l], label = "forward_rxn", weight = int(10*0.3))
            #print("ADD EDGE")
            l+=1
        k+=1
    
    j=0
    #print(i/len(orfs))
    #nx.write_graphml(G,"metabolism_test.xml")
    #print (str(i /len(orfs)) + " " + str(i) + " " + str(len(orfs)), end='\r')
    i+=1
G.remove_node("PROTON")
G.remove_node("WATER")
G.remove_node("ATP")
G.remove_node("ADP")
G.remove_node("PPI")
G.remove_node("Pi")
G.remove_node("Protein-L-serine-or-L-threonine")
G.remove_node("Protein-Ser-or-Thr-phosphate")
G.remove_node("AMP")
G.remove_node("NAD")
G.remove_node("NADH")
G.remove_node("CO-A")
G.remove_node("NADP")
G.remove_node("NADPH")
G.remove_node("CARBON-DIOXIDE")
G.remove_node("GLT")
G.remove_node("S-ADENOSYLMETHIONINE")
G.remove_node("OXYGEN-MOLECULE")
G.remove_node("ACETYL-COA")
G.remove_node("AMMONIUM")
G.remove_node("ADENOSYL-HOMO-CYS")
G.remove_node("Nucleoside-Triphosphates")
G.remove_node("Peptides-holder")
G.remove_node("RNA-Holder")
G.remove_node("Cytochromes-C-Oxidized")
G.remove_node("Cytochromes-C-Reduced")
G.remove_node("GDP")
G.remove_node("Ubiquitin-C-Terminal-Glycine")
G.remove_node("General-Protein-Substrates")

i = 0

while i < len(orfs):

    j=0
    k = 0
    while k < len(interactors[i]):

        if interactors_type[i][k] == "Positive Genetic":
            G.add_edge(orfs[i], interactors[i][k], label = interactors_type[i][k], weight = int(10))
          
        k+=1
    i+=1
    #print (str(i /len(orfs)) + " " + str(i) + " " + str(len(orfs)), end='\r')



nx.write_graphml(G,"citrate_posinteractions.xml")

In [ ]:
nx.dijkstra_path_length(G1, 'TRS120', 'CIT')

In [ ]:
G = nx.Graph()
G1 = nx.Graph()
i = 0
G.add_node('CIT', form = 'compound', distance = int(0))
while i < len(orfs):
    try:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = int(10*round(nx.dijkstra_path_length(G1, orfs[i], 'CIT'), 1)))
    except:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = -1)
        
    k = 0
    while k < len(left[i]):
        l = 0
        while l < len(left[i][k]):
            try:
                G.add_node(left[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, left[i][k][l], 'CIT'),1)))
            except:
                G.add_node(left[i][k][l], form = 'compound', distance = -1)
                
            G.add_edge(orfs[i], left[i][k][l], label = "backward_rxn", weight = int(10*0.5))
            #print("add edge")
            l+=1
        k+=1
    
          
    k = 0
    while k < len(right[i]):
        l = 0
        while l < len(right[i][k]):
            try:
                G.add_node(right[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, right[i][k][l], 'CIT'),1)))
            except:
                G.add_node(right[i][k][l], form = 'compound', distance = -1)
            G.add_edge(orfs[i], right[i][k][l], label = "forward_rxn", weight = int(10*0.5))
            #print("ADD EDGE")
            l+=1
        k+=1
    
    j=0
    #print(i/len(orfs))
    #nx.write_graphml(G,"metabolism_test.xml")
    print (i /len(orfs), end='\r')
    i+=1
G.remove_node("PROTON")
G.remove_node("WATER")
G.remove_node("ATP")
G.remove_node("ADP")
G.remove_node("PPI")
G.remove_node("Pi")
G.remove_node("Protein-L-serine-or-L-threonine")
G.remove_node("Protein-Ser-or-Thr-phosphate")
G.remove_node("AMP")
G.remove_node("NAD")
G.remove_node("NADH")
G.remove_node("CO-A")
G.remove_node("NADP")
G.remove_node("NADPH")
G.remove_node("CARBON-DIOXIDE")
G.remove_node("GLT")
G.remove_node("S-ADENOSYLMETHIONINE")
G.remove_node("OXYGEN-MOLECULE")
G.remove_node("ACETYL-COA")
G.remove_node("AMMONIUM")
G.remove_node("ADENOSYL-HOMO-CYS")
G.remove_node("Nucleoside-Triphosphates")
G.remove_node("Peptides-holder")
G.remove_node("RNA-Holder")
G.remove_node("Cytochromes-C-Oxidized")
G.remove_node("Cytochromes-C-Reduced")
G.remove_node("GDP")
G.remove_node("Ubiquitin-C-Terminal-Glycine")
G.remove_node("General-Protein-Substrates")

i = 0

while i < len(orfs):
    k = 0
    while k < len(interactors[i]):
        if interactors_type[i][k] == "Positive Genetic":
            G.add_edge(names[i], interactors[i][k], label = interactors_type[i][k], weight = int(10))
 
        k+=1

    i+=1
    print (i /len(orfs))

import copy
G1 = copy.deepcopy(G)


G = nx.Graph()
i = 0
G.add_node('CIT', form = 'compound', distance = int(0))
while i < len(orfs):
    try:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = int(10*round(nx.dijkstra_path_length(G1, orfs[i], 'CIT'), 1)))
    except:
        G.add_node(orfs[i], form = 'orf', name = names[i], distance = -1)
    k = 0
    while k < len(left[i]):
        l = 0
        while l < len(left[i][k]):
            try:
                G.add_node(left[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, left[i][k][l], 'CIT'),1)))
            except:
                G.add_node(left[i][k][l], form = 'compound', distance = -1)
            G.add_edge(orfs[i], left[i][k][l], label = "backward_rxn", weight = 10*0.5)
            #print("add edge")
            l+=1
        k+=1
    
          
    k = 0
    while k < len(right[i]):
        l = 0
        while l < len(right[i][k]):
            try:
                G.add_node(right[i][k][l], form = 'compound', distance = int(10*round(nx.dijkstra_path_length(G1, right[i][k][l], 'CIT'),1)))
            except:
                G.add_node(right[i][k][l], form = 'compound', distance = -1)
            G.add_edge(orfs[i], right[i][k][l], label = "forward_rxn", weight = int(10*0.5))
            #print("ADD EDGE")
            l+=1
        k+=1
    
    j=0
    #print(i/len(orfs))
    #nx.write_graphml(G,"metabolism_test.xml")
    print (i /len(orfs), end='\r')
    i+=1
G.remove_node("PROTON")
G.remove_node("WATER")
G.remove_node("ATP")
G.remove_node("ADP")
G.remove_node("PPI")
G.remove_node("Pi")
G.remove_node("Protein-L-serine-or-L-threonine")
G.remove_node("Protein-Ser-or-Thr-phosphate")
G.remove_node("AMP")
G.remove_node("NAD")
G.remove_node("NADH")
G.remove_node("CO-A")
G.remove_node("NADP")
G.remove_node("NADPH")
G.remove_node("CARBON-DIOXIDE")
G.remove_node("GLT")
G.remove_node("S-ADENOSYLMETHIONINE")
G.remove_node("OXYGEN-MOLECULE")
G.remove_node("ACETYL-COA")
G.remove_node("AMMONIUM")
G.remove_node("ADENOSYL-HOMO-CYS")
G.remove_node("Nucleoside-Triphosphates")
G.remove_node("Peptides-holder")
G.remove_node("RNA-Holder")
G.remove_node("Cytochromes-C-Oxidized")
G.remove_node("Cytochromes-C-Reduced")
G.remove_node("GDP")
G.remove_node("Ubiquitin-C-Terminal-Glycine")
G.remove_node("General-Protein-Substrates")

i = 0

while i < len(orfs):

    j=0
    k = 0
    while k < len(interactors[i]):

        if interactors_type[i][k] == "Positive Genetic":
            G.add_edge(orfs[i], interactors[i][k], label = interactors_type[i][k], weight = int(10))
          
        k+=1
    i+=1
    print (i /len(orfs))



nx.write_graphml(G,"citrate_posinteractions_equalweight.xml")

In [ ]:
for value in G.nodes:
    try:
        G.nodes[value]['distance'] = int(10*round(nx.dijkstra_path_length(G, value, 'CIT'),1))
    except:
        G.nodes[value]['distance'] = -1

In [ ]:
nx.write_graphml(G,"citrate_posinteractions_equalweight.xml")

In [ ]:
orfs = pd.read_csv('final_citrate_pos_interactions_orfs.csv')
sns.countplot(data = orfs, x = "distance")
plt.savefig('distance_chart.svg'